# Sequential Chats and Customer Onboarding

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

config_list = [
  {
    "model": "llama-3.1-70b-versatile",
    "base_url": "https://api.groq.com/openai/v1",
    "api_key": os.getenv('GROQ_API_KEY'),
    "price": [0, 0],
  },
  # {"model": "gpt-3.5-turbo", "api_key": os.environ["OPENAI_API_KEY"]} 
]
llm_config = {"config_list": config_list}

In [2]:
from autogen import ConversableAgent

## Agents for Customer Onboarding

```mermaid
---
title: The Agents' Relationship
---
sequenceDiagram
    autonumber

    actor Info_Agent
    actor Customer
    actor Topic_Agent
    actor Engagement_Agent
    
    Info_Agent->>+Customer: name and location?
    Customer-->>-Info_Agent: ...
    
    Info_Agent -->> Engagement_Agent: name, location
    
    
    Topic_Agent->>+Customer: topics?
    Customer-->>-Topic_Agent: ...
    
    Topic_Agent -->> Engagement_Agent: topics
    
    Engagement_Agent->>+Customer: ...
    Customer->>-Engagement_Agent: ...

```

In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [7]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "last_msg",# if it is 1, just use the "last_msg", otherwise it might not retrieve the carryover
        "max_turns": 1, 
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]
# https://microsoft.github.io/autogen/docs/tutorial/conversation-patterns/#sequential-chats

## Start the onboarding process

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/Users/yanmeng/anaconda3/envs/autogen/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Replying as customer_proxy_agent. Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Yan Meng


customer_proxy_agent (to Onboarding Personal Information Agent):

Yan Meng

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

So, your name is Yan Meng. Could you please tell me where you are located (city, state, country)?

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Xi'an


customer_proxy_agent (to Onboarding Personal Information Agent):

Xi'an

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{'name': 'Yan Meng', 'location': 'Xi'an'}

--------------------------------------------------------------------------------


Replying as customer_proxy_agent. Provide feedback to Onboarding Topic preference Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  Sport


customer_proxy_agent (to Onboarding Topic preference Agent):

Sport

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
customer_proxy_agent (to Customer Engagement Agent):

Let's find something fun to read.
Context: 
{'name': 'Yan Meng', 'location': 'Xi'an'}
Sport

--------------------------------------------------------------------------------
Customer Engagement Agent (to customer_proxy_agent):

Ni hao, Yan Meng!

I heard you're from Xi'an, the city of the Terracotta Warriors! Did you know that Xi'an is also famous for its traditional Chinese sports, like Cuju ( ancient Chinese football) and Dragon Boat Racing?

Speaking of sports, I've got a fun fact for you: Did you know that the oldest known evidence of a sport being played in China dates back to the Shang Dynasty (

In [9]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'Yan Meng', 'location': 'Xi'an'}


Sport


There is no conversation to summarize.


